# Import potrebných knižníc

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import numpy as np
import os
import glob

# Odstránenie hviezd z BU

Hviezdy na nočných snímkach by mohli ovplivniť predikciu výskytu Airglow a teda sme sa rozhodli zmierniť ich dopad

Na to sme využili Gausovské rozostrenie ktoré ich rozmaže 

In [ ]:
#vytvorenie potrebných priečinkov
source_dir = 'BU_rio_grande'
destination_dir = 'BU_rio_grande_stars'
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

image_files = glob.glob(os.path.join(source_dir, '*.png'))

In [58]:
#iterácia pre každý obrázok
for image_path in image_files:
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    has_alpha = image.shape[2] == 4
    #získanie rozmerov obrázku
    rows, cols = image.shape[:2]
    #vytvorenie masky
    mask = np.zeros((rows, cols), dtype=np.uint8)
    #stred obrázku a rádius
    center = (cols // 2, rows // 2)
    radius = min(center[0], center[1], cols - center[0], rows - center[1])
    cv2.circle(mask, center, radius, 255, -1)

    if has_alpha:
        color_channels = image[..., :3] # Oddelenie farebných kanálov
        alpha_channel = image[..., 3] # Oddelenie alfa kanála
    else:
        color_channels = image
        alpha_channel = None
    #aplikovanie Gausovského rozmazania
    blurred_color_channels = cv2.GaussianBlur(color_channels, (7, 7), 0)
    blurred_image = np.where(mask[:, :, None] == 255, blurred_color_channels, color_channels)

    if has_alpha:
        result_image = cv2.merge((*cv2.split(blurred_image), alpha_channel))
    else:
        result_image = blurred_image

    #uloženie do priečinku s odstránenými zviezdami
    filename = os.path.basename(image_path)
    save_path = os.path.join(destination_dir, filename)
    cv2.imwrite(save_path, result_image)